In [42]:
from bs4 import BeautifulSoup
import urllib.request 
import re
import pandas as pd
from collections import Counter

In [19]:
url = input('Enter your link:')

def crawl_content(url):
	response = urllib.request.urlopen(url) 
	html = response.read()
	soup = BeautifulSoup(html,"html.parser")
	text = soup.get_text(strip=True)
	pattern = "(\(VTC News\) -)(.+)(\(adsvtc)?"
	a = re.search(pattern, text)
	if a == None:
		return
	else:
		text = a.group(2).split('(adsvtc = window.adsvtc || []).push();')[0]
		text = text.lower()
		text = re.sub('[0-9\!\@\#\$\%\^\&\*\(\)\_\.\+\-\,\?\:\/\"\'\<\>\=\~\[\]\{\}\|\–]+',' ',text)
		print('crawl success')
		return text
print(crawl_content(url))

Enter your link:https://vtc.vn/gap-co-giao-voi-bai-tho-dang-tao-song-tren-mang-xa-hoi-ve-day-hoc-thoi-covid-ar615497.html
crawl success
những ngày qua  trên mạng xã hội chuyền tay nhau một bài thơ thu hút sự chú ý của nhiều thế hệ học trò cũng như các vị phụ huynh học sinh cô ngô hương  giáo viên trường thcs lê lợi đã chia sẻ một bài thơ về  vị cứu nhân  giúp cô trò trường thcs lê lợi  hưng yên kết thúc sớm  năm học covid  vượt lên mọi khó khăn trong quá trình dạy học ấn tượng đầu tiên chính là nét chữ đẹp rắn rỏi vừa mềm mại điển hình của một giáo viên dạy văn  những dòng tâm sự lắng đọng về một “thời covid ” bài thơ của cô giáo nhận được nhiều lượt chia sẻ và yêu thích của cộng đồng mạng như   thơ hay quá cô giáo ơi   cô giáo dạy văn có khác   cô hương tuyệt vời lắm lắm với bài thơ “gây sóng” của mình  facebook cá nhân của cô ngô hương được các thế hệ học sinh  và cả phụ huynh tìm kiếm  thật thú vị khi tìm thấy cô trên không gian mạng  để biết được những câu chuyện thường ngày ấm áp 

In [ ]:
with open('corpus.csv', 'w', newline='', encoding='utf-8') as file:
	writer = csv.writer(file)
	writer.writerow(["content", "label"])
	i = 0
	for f in topics:
		j = 1
		urls = open(f,'r')
		for url in urls:
			url = url.strip()
			text = crawl_content(url)
			if text==None:
				continue
			else:
				writer.writerow([text,i])
				j = j+1
		print('done topic num ',i)
		i = i+1
		print('topic ',i,' has ',j, 'content!')

In [25]:
df = pd.read_csv('corpus.csv',encoding='utf-8')

In [26]:
print(df.info)

<bound method DataFrame.info of                                                 content  label
0     hàng loạt cổ phiếu lớn chìm trong sắc đỏ khiến...      0
1     trong kỳ điều chỉnh chiều nay   giá xăng e  ro...      0
2     theo hiệp hội quảng cáo  nghị định  nđ cp có n...      0
3     nhiều nhà đầu tư tiền ảo yêu cầu elon musk ngừ...      0
4     ngày   tỷ phú jeff bezos tuyên bố sẽ rút khỏi ...      0
...                                                 ...    ...
6019  người dân vùng altai ở nga tin rằng xác ướp từ...      4
6020  hôm nay  công ty bảo hiểm nhân thọ bidv metlif...      4
6021  một nghiên cứu mới đây của các nhà khoa học tớ...      4
6022  ngày   vinai research công bố nghiên cứu thành...      4
6023  con sói xám châu âu đơn độc di chuyển xung qua...      4

[6024 rows x 2 columns]>


In [27]:
print(df.shape)
print(df.info())
#dataset inclues 2 columns, 1st is content of news on website vtcnews, 2nd is label for the content.
#5 topics, economic = 0, sport = 1, education = 2, heath = 3, and science-technology = 4, each topic includes about 1200 samples

(6024, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6024 entries, 0 to 6023
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  6024 non-null   object
 1   label    6024 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 94.2+ KB
None


In [28]:
print(df.describe()) # statics about label column of text content, mean ~2.0, that is data distributes uniformly

             label
count  6024.000000
mean      2.009794
std       1.409181
min       0.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       4.000000


In [29]:
def token(row):
    row = re.split('\...|\n|\s|\"|\'|\,|\.', str(row))
    row = list(filter(None, row))
    return row

df['content'] = df['content'].apply(token)
print(df['content'])

0       [hàng, loạt, cổ, phiếu, lớn, chìm, trong, sắc,...
1       [trong, kỳ, điều, chỉnh, chiều, nay, giá, xăng...
2       [theo, hiệp, hội, quảng, cáo, nghị, định, nđ, ...
3       [nhiều, nhà, đầu, tư, tiền, ảo, yêu, cầu, elon...
4       [ngày, tỷ, phú, jeff, bezos, tuyên, bố, sẽ, rú...
                              ...                        
6019    [người, dân, vùng, altai, ở, nga, tin, rằng, x...
6020    [hôm, nay, công, ty, bảo, hiểm, nhân, thọ, bid...
6021    [một, nghiên, cứu, mới, đây, của, các, nhà, kh...
6022    [ngày, vinai, research, công, bố, nghiên, cứu,...
6023    [con, sói, xám, châu, âu, đơn, độc, di, chuyển...
Name: content, Length: 6024, dtype: object


In [39]:
def most_frequent(List):
    return max(set(List), key = List.count)
def least_frequent(List):
    return min(set(List), key = List.count)

df["size"] = df['content'].apply(len)
df["most"] = df['content'].apply(most_frequent)
df["least"] = df['content'].apply(least_frequent)

In [40]:
print(df.head(10))

                                             content  label  size    most  \
0  [hàng, loạt, cổ, phiếu, lớn, chìm, trong, sắc,...      0   376    điểm   
1  [trong, kỳ, điều, chỉnh, chiều, nay, giá, xăng...      0   245    đồng   
2  [theo, hiệp, hội, quảng, cáo, nghị, định, nđ, ...      0   823   quảng   
3  [nhiều, nhà, đầu, tư, tiền, ảo, yêu, cầu, elon...      0   528    tiền   
4  [ngày, tỷ, phú, jeff, bezos, tuyên, bố, sẽ, rú...      0   259  amazon   
5  [chuỗi, cung, ứng, chuỗi, giá, trị, toàn, cầu,...      0   431   doanh   
6  [ngày, đại, diện, tập, đoàn, tài, chính, m...      0   708      mb   
7  [đầu, ngày, những, lô, vải, thiều, đầu, tiên, ...      0   543     vải   
8  [từ, đầu, năm, đến, nay, giá, lợn, hơi, liên, ...      0   927     giá   
9  [giá, mỗi, đồng, tiền, ảo, đã, tăng, mạnh, chỉ...      0   281     usd   

   least  
0     từ  
1     dự  
2     vì  
3    hãi  
4     vì  
5   hiệu  
6     vì  
7  thuận  
8  nhuận  
9   rằng  


In [46]:
print(df.describe())

             label         size
count  6024.000000  6024.000000
mean      2.009794   569.722112
std       1.409181   363.919595
min       0.000000    10.000000
25%       1.000000   324.000000
50%       2.000000   454.000000
75%       3.000000   731.000000
max       4.000000  5161.000000


In [44]:
print(df[df['label']==3]['most'])

3529      bệnh
3530      tính
3531    bệnh
3532      bệnh
3533       gốc
         ...  
4854        bộ
4855        cư
4856        có
4857     người
4858       hcm
Name: most, Length: 1330, dtype: object


In [43]:
count = Counter(df['content'][0])
print(count)

Counter({'điểm': 19, 'giảm': 14, 'tăng': 10, 'cổ': 8, 'phiếu': 8, 'mã': 8, 'vn': 7, 'hàng': 6, 'trong': 6, 'index': 6, 'phiên': 6, 'số': 6, 'sắc': 5, 'chỉ': 5, 'xuống': 5, 'giá': 5, 'loạt': 4, 'lớn': 4, 'đỏ': 4, 'khiến': 4, 'hơn': 4, 'và': 4, 'giao': 4, 'với': 4, 'thị': 4, 'trường': 4, 'chìm': 3, 'lực': 3, 'bán': 3, 'hose': 3, 'ngân': 3, 'lên': 3, 'như': 3, 'sàn': 3, 'dịch': 3, 'tiếp': 2, 'áp': 2, 'trên': 2, 'hiện': 2, 'một': 2, 'các': 2, 'duy': 2, 'trì': 2, 'được': 2, 'h': 2, 'đứng': 2, 'sáng': 2, 'đã': 2, 'khi': 2, 'fpt': 2, 'hpg': 2, 'msn': 2, 'đồng': 2, 'có': 2, 'ảnh': 2, 'mạnh': 2, 'toàn': 2, 'chứng': 2, 'khoán': 2, 'thép': 2, 'bất': 2, 'động': 2, 'thanh': 2, 'khoản': 2, 'eib': 2, 'dòng': 2, 'về': 1, 'sát': 1, 'mốc': 1, 'sau': 1, 'liên': 1, 'thiết': 1, 'lập': 1, 'đỉnh': 1, 'mới': 1, 'diện': 1, 'rộng': 1, 'xuất': 1, 'ngay': 1, 'từ': 1, 'sớm': 1, 'dịchchứng': 1, 'khoánsáng': 1, 'nay': 1, 'tuy': 1, 'nhiên': 1, 'nhờ': 1, 'sự': 1, 'bứt': 1, 'phá': 1, 'của': 1, 'nên': 1, 'vẫn': 1, 'xanh